# Model Selection
### Imports

In [41]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
# from xgboost import XGBRegressor


### Import Data
Import the data generated by the data_clean.ipynb notebook

In [42]:
# Read data from data cleaning
total_data_df = pd.read_csv('output.csv')


## Parameter Selection + Regularization
We will use Standardization (Z-score normalization), which scales the features so that they have a mean of 0 and a standard deviation of 1. It subtracts the mean of each feature and divides by its standard deviation. Standardization can help algorithms converge faster and perform better, especially for algorithms that rely on distance metrics or gradient descent.

In [43]:
from sklearn.preprocessing import StandardScaler

# Select features and target
# features = ['Pos_PG', 'Pos_SG', 'Pos_SF', 'Pos_PF', 'Pos_C', 'G', 'GS', 'MP', 'FG', 'FG%', '3P', '3P%', '2P', '2P%', 'eFG%', 'ORB', 'DRB', 'AST', 'BLK', 'PTS', 'PER', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Tm_Rcrd', 'MVP_strk']
# features = ['Pos_PG', 'Pos_SG', 'Pos_SF', 'Pos_PF', 'Pos_C', 'G', 'GS', 'MP', 'FG', 'FG%', '3P', '3P%', '2P', '2P%', 'eFG%', 'ORB', 'DRB', 'AST', 'BLK', 'PTS', 'PER', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Tm_Rcrd']
features = ['Age' ,'G', 'GS', 'MP', 'FG', 'FG%', '3P', '3P%', 'eFG%', 'ORB', 'DRB', 'AST', 'PTS', 'PER', 'FTr', 'USG%', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'VORP', 'Tm_Rcrd']
target = 'MVP_Shr'

# scale the features: uncomment the following lines to scale the features
# scaler = StandardScaler()
# total_data_df[features] = scaler.fit_transform(total_data_df[features])

# total_data_df.head(10)


## Experimenting with different models
We're going to have to try a lot of different models and parameter tuning via K-Fold Cross Validation, where the set we leave out (for validation), will be a set corresponding to a particular season

In [44]:
season_list = [f"{year}_{year + 1 - 2000:02}" for year in range(2000, 2023)]

def model_testing(model):
    mvp_list = []
    mse_sum = 0
    r2_square_sum = 0
    for season in season_list:
        # Split data into training and testing sets
        X_train = total_data_df[total_data_df['Szn'] != season][features]
        X_test = total_data_df[total_data_df['Szn'] == season][features]
        y_train = total_data_df[total_data_df['Szn'] != season][target]
        y_test = total_data_df[total_data_df['Szn'] == season][target]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        # add the top candidates to the prediction
        predictions = pd.DataFrame({'Player': total_data_df[total_data_df['Szn'] == season]['Plyr'], 'Predicted MVP Share': y_pred})
        mvp = predictions.sort_values(by='Predicted MVP Share', ascending=False).head(1)['Player'].values[0]
        # print(mvp)
        # print the type of mvp variable
        # print(type(mvp))

        mvp_list.append(mvp)         # values[0] is to select the value since dataframes are by default indexed
        
        # predictions = pd.DataFrame({'Player': total_data_df[total_data_df['Szn'] == season]['Plyr'], 'Predicted MVP Share': y_pred})
        # print(f'MVP Predictions for the {season} season:')
        # print(predictions.sort_values(by='Predicted MVP Share', ascending=False).head(1))
        # print('\n')
        #evaluate MSE and R^2
        mse = mean_squared_error(y_test, y_pred)
        r_squared = r2_score(y_test, y_pred)
        mse_sum += mse
        r2_square_sum += r_squared
        print(f'{season} : MSE and R-squared:', mse, r_squared)

    mse_avg = mse_sum / len(season_list)
    r_squared_avg = r2_square_sum / len(season_list)

    print('AVG MSE and R-squared:', mse_avg, r_squared_avg)
    # print(mvp_list)
    return mvp_list

In [45]:
# list of all models, and their string representation

models = {
    'Linear Regression': LinearRegression(),                # 0.5652173913043478
    'Lasso': Lasso(alpha=0.01),                              # 0.5652173913043478
    'Ridge': Ridge(alpha=0.1),                              # 0.5652173913043478
    'Bagging Regression': BaggingRegressor(n_estimators=200), # 0.5652173913043478
    'Decision Tree': DecisionTreeRegressor(max_features=30),               # 0.4782608695652174
    'Random Forest': RandomForestRegressor(n_estimators=50, max_features=5), # 0.6521739130434783
    'Gradient Boosting': GradientBoostingRegressor(max_depth=5, learning_rate=0.21, n_estimators=50),       # 0.5652173913043478
    'AdaBoost': AdaBoostRegressor(learning_rate= 0.5, n_estimators=50),    # 0.6086956521739131
    'Support Vector Machine': SVR(),                        # 0.6956521739130435
    # 'Elastic Net': ElasticNet(),                            # don't even try this
    # 'Bayesian Ridge': BayesianRidge(),                      # 0.4782608695652174
    # 'K Nearest Neighbors': KNeighborsRegressor(),           # 0.5652173913043478
    # 'Multi-layer Perceptron': MLPRegressor(),               # sucks
    # 'XGBoost': XGBRegressor()                               # 0.5652173913043478
}

results = []

# Test each model
for model_name, model_object in models.items():
    print(f'Testing {model_name}')
    model_result = model_testing(model_object)
    # print(model_result)
    results.append(model_result)

# Make a dataframe, where each column is a model, and each row is a season
results_df = pd.DataFrame()

mvp_winners = ['Allen Iverson', 'Tim Duncan', 'Tim Duncan', 'Kevin Garnett', 'Steve Nash', 'Steve Nash', 'Dirk Nowitzki', 'Kobe Bryant', 'LeBron James', 'LeBron James', 'Derick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo', 'Giannis Antetokounmpo', 'Nikola Jokić', 'Nikola Jokić', 'Joel Embiid']
results_df['Season'] = season_list
results_df['Truth'] = mvp_winners

for index, (key, value) in enumerate(models.items()):
    results_df[key] = results[index]



Testing Linear Regression
2000_01 : MSE and R-squared: 0.01704639669386141 0.1696450870498729
2001_02 : MSE and R-squared: 0.01529569603380658 0.23256457030719746
2002_03 : MSE and R-squared: 0.009814494563609201 0.46883396134457744
2003_04 : MSE and R-squared: 0.011672362482720888 0.414759617082304
2004_05 : MSE and R-squared: 0.02136455747994239 -0.06804230440251557
2005_06 : MSE and R-squared: 0.01351462299571637 0.18101504912313526
2006_07 : MSE and R-squared: 0.015035493252589831 0.3070926737466384
2007_08 : MSE and R-squared: 0.012737609935446677 0.38923346815156357
2008_09 : MSE and R-squared: 0.009358195682169558 0.5775319332469198
2009_10 : MSE and R-squared: 0.010996296433771932 0.4913062619558395
2010_11 : MSE and R-squared: 0.015177319662861247 0.30374932059116144
2011_12 : MSE and R-squared: 0.019183886496839296 0.1861688037537319
2012_13 : MSE and R-squared: 0.009582976626218594 0.5537396604393198
2013_14 : MSE and R-squared: 0.012457437868736675 0.47112828536541473
2014_

In [46]:
results_df

,Season,Truth,Linear Regression,Lasso,Ridge,Bagging Regression,Decision Tree,Random Forest,Gradient Boosting,AdaBoost,Support Vector Machine
0,2000_01,Allen Iverson,Shaquille O'Neal,Shaquille O'Neal,Shaquille O'Neal,Shaquille O'Neal,John Stockton,Shaquille O'Neal,Shaquille O'Neal,Shaquille O'Neal,Shaquille O'Neal
1,2001_02,Tim Duncan,Tim Duncan,Tim Duncan,Tim Duncan,Tim Duncan,Shaquille O'Neal,Tim Duncan,Shaquille O'Neal,Tim Duncan,Tim Duncan
2,2002_03,Tim Duncan,Tracy McGrady,Tracy McGrady,Tracy McGrady,Tracy McGrady,Tim Duncan,Tracy McGrady,Kobe Bryant,Tracy McGrady,Tim Duncan
3,2003_04,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett
4,2004_05,Steve Nash,Kevin Garnett,Kevin Garnett,Kevin Garnett,Kevin Garnett,Jermaine O'Neal,Kevin Garnett,Kevin Garnett,Dirk Nowitzki,Dirk Nowitzki
5,2005_06,Steve Nash,LeBron James,LeBron James,LeBron James,Dirk Nowitzki,Dirk Nowitzki,Dirk Nowitzki,Dirk Nowitzki,Dirk Nowitzki,Dirk Nowitzki
6,2006_07,Dirk Nowitzki,LeBron James,Dirk Nowitzki,LeBron James,Steve Nash,Tim Duncan,Steve Nash,Steve Nash,Steve Nash,Dirk Nowitzki
7,2007_08,Kobe Bryant,LeBron James,LeBron James,LeBron James,Chris Paul,Deron Williams,Chris Paul,Chris Paul,Chris Paul,Chris Paul
8,2008_09,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James
9,2009_10,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James,LeBron James


### Accuracy Calculation

In [39]:
# Accuracy Calculation (correct/23)
model_names = list(models.keys())

for i in range(len(results)):
    correct = 0
    for j in range(len(results[0])):
        if results[i][j] == mvp_winners[j]:
            correct += 1
    print(f"{model_names[i]}            : ", correct/23)


Linear Regression            :  0.4782608695652174
Lasso            :  0.6521739130434783
Ridge            :  0.4782608695652174
Bagging Regression            :  0.5217391304347826
Random Forest            :  0.6086956521739131
Gradient Boosting            :  0.5217391304347826
AdaBoost            :  0.6086956521739131
Support Vector Machine            :  0.7391304347826086
